In [18]:
# edit from news classifier from here: https://www.kaggle.com/code/dimitriosroussis/news-articles-classification-tf-idf-voting/notebook

import json
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from timeit import default_timer as timer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords

np.random.seed(2023)

In [2]:
!#pip uninstall matplotlib
#!pip install matplotlib

'#pip' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
#dataset read in 
with open('dataset/evidence.json') as json_file:
    evidence = json.load(json_file)

with open('dataset/dev-claims.json') as json_file:
    dev_claim = json.load(json_file)

with open('dataset/train-claims.json') as json_file:
    train = json.load(json_file)

with open('dataset/test-claims-unlabelled.json') as json_file:
    test = json.load(json_file)


In [16]:
print(len(evidence.keys()))
evi_used_train = []
evi_used_dev = []
for key in train.keys():
    evi_used_train += train[key]['evidences']
evi_used_train = list(set(evi_used_train))

for key in dev_claim.keys():
    evi_used_dev += dev_claim[key]['evidences']
evi_used_dev = list(set(evi_used_dev))

print(len(evi_used_train))
print(len(evi_used_dev))
print(len(evi_used_train)/len(evidence.keys()))
print(len(evi_used_dev)/len(evidence.keys()))




common = len(list(set(evi_used_train).intersection(evi_used_dev)))
print(common/len(evi_used_train))
print(common/len(evi_used_dev))
print(common/len(evidence.keys()))


1208827
3121
463
0.0025818417358315126
0.0003830159319737233
0.045177827619352774
0.3045356371490281
0.00011664200088184661


In [20]:
test_da = []
for key in test.keys():
    test_da.append(test[key]['claim_text'])
#print(len(test_da))
dataset = {'text':test_da}
data_test = pd.DataFrame.from_dict(dataset)
X_test = data_test['text'].values
    

In [21]:
#pre processing, without evicence
label = []
claim_text = []
for key in train.keys():
    label.append(train[key]['claim_label'])
    claim_text.append(train[key]['claim_text'])

dataset = {'text':claim_text, 'label':label}
data = pd.DataFrame.from_dict(dataset)

data.head()

,text,label
0,Not only is there no scientific evidence that ...,DISPUTED
1,El Niño drove record highs in global temperatu...,REFUTES
2,"In 1946, PDO switched to a cool phase.",SUPPORTS
3,Weather Channel co-founder John Coleman provid...,DISPUTED
4,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO


In [22]:
#pre processing, with evicence and only evidence id
label = []
text = []
for key in train.keys():
    local = train[key]["claim_text"] + ' '
    evi = [i for i in (train[key]['evidences'])] 
    local += ' '.join(evi)
    label.append(train[key]['claim_label'])
    text.append(local)

dataset = {'text':text, 'label':label}
data_evid = pd.DataFrame.from_dict(dataset)

data_evid.head()



,text,label
0,Not only is there no scientific evidence that ...,DISPUTED
1,El Niño drove record highs in global temperatu...,REFUTES
2,"In 1946, PDO switched to a cool phase. evidenc...",SUPPORTS
3,Weather Channel co-founder John Coleman provid...,DISPUTED
4,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO


In [23]:
#pre processing, with evicence text
label = []
text = []
for key in train.keys():
    local = train[key]["claim_text"] + ' '
    evi = [i for i in (train[key]['evidences'])] 
    evi_text = []
    for id in evi:
        evi_text.append(evidence[id])
    local += ' '.join(evi_text)
    label.append(train[key]['claim_label'])
    text.append(local)



dataset = {'text':text, 'label':label}
data_evte = pd.DataFrame.from_dict(dataset)

data_evte.head()
print(data_evte.text[0])

Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life. At very high concentrations (100 times atmospheric concentration, or greater), carbon dioxide can be toxic to animal life, so raising the concentration to 10,000 ppm (1%) or higher for several hours will eliminate pests such as whiteflies and spider mites in a greenhouse. Plants can grow as much as 50 percent faster in concentrations of 1,000 ppm CO 2 when compared with ambient conditions, though this assumes no change in climate and no limitation on other nutrients. Higher carbon dioxide concentrations will favourably affect plant growth and demand for water.


In [24]:
#checking wether 0R is a good choice(Actually not bad)
rel_freq = data.groupby('label').text.count() / len(data)
print(rel_freq)

label
DISPUTED           0.100977
NOT_ENOUGH_INFO    0.314332
REFUTES            0.162052
SUPPORTS           0.422638
Name: text, dtype: float64


In [25]:
def get_metrics(y_true, y_pred, metrics):
    metrics[0] += accuracy_score(y_true, y_pred)
    metrics[1] += precision_score(y_true, y_pred, average='macro')
    metrics[2] += recall_score(y_true, y_pred, average='macro')
    metrics[3] += f1_score(y_true, y_pred, average='macro')
    return metrics

def evaluate_classifier(clf, kfold, X, y, vectorizer):
    metrics = np.zeros(4)
    start = timer()
    for train, cv in kfold.split(X, y):
        X_train, X_cv = X[train], X[cv]
        y_train, y_cv = y[train], y[cv]
        X_train_gen = [x for x in X_train]
        #vectorizer.fit(X_train_gen)
        X_train_vec = vectorizer.transform(X_train_gen)
        clf.fit(X_train_vec, y_train)
        X_cv_gen = [x for x in X_cv]
        X_cv_vec = vectorizer.transform(X_cv_gen)
        y_pred = clf.predict(X_cv_vec)
        metrics = get_metrics(y_cv, y_pred, metrics)
    dt = timer() - start
    metrics = metrics * 100 / 5
    print('Evaluation of classifier finished in {:.2f} s \n'
          'Average accuracy: {:.2f} % \n'
          'Average precision: {:.2f} % \n'
          'Average recall: {:.2f} % \n'
          'Average F-Measure: {:.2f} % \n'
          .format(dt, metrics[0], metrics[1],
                  metrics[2], metrics[3]))

In [26]:
kf = KFold(n_splits=5, shuffle=True, random_state=56)

#stop_words = list(ENGLISH_STOP_WORDS)
stop_words = stopwords.words('english')
#print(stopwords)
tfidf = TfidfVectorizer(stop_words = stop_words)
#stop_words = stop_words
trainner = []

for i in evidence.keys():
    trainner.append(evidence[i])
for key in train.keys():
    trainner.append(train[key]['claim_text'])
for key in test.keys():
    trainner.append(test[key]['claim_text'])
#c_t = [x for x in text] 
#trainner += c_t
tfidf.fit(raw_documents = trainner)




TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [27]:
X = data['text'].values
y = data['label'].values


X_train_cv, X_test, y_train_cv, y_test = train_test_split(X, y, test_size=0.2, 
                                                          random_state=56)

In [65]:
svm = LinearSVC(tol=1e-05, max_iter=1500)
evaluate_classifier(svm, kf, X_train_cv, y_train_cv, tfidf)

Evaluation of classifier finished in 0.31 s 
Average accuracy: 45.01 % 
Average precision: 38.26 % 
Average recall: 33.78 % 
Average F-Measure: 33.32 % 



In [66]:
ridge = RidgeClassifier(alpha=0.8, tol=1e-05)
evaluate_classifier(ridge, kf, X_train_cv, y_train_cv, tfidf)

Evaluation of classifier finished in 2.65 s 
Average accuracy: 45.62 % 
Average precision: 35.31 % 
Average recall: 33.01 % 
Average F-Measure: 31.78 % 



In [72]:


n_neight = 21
knn = KNeighborsClassifier(n_neighbors = n_neight, n_jobs=-1, weights = 'distance')
evaluate_classifier(knn, kf, X_train_cv, y_train_cv, tfidf)

c:\Users\a1360\anaconda3\envs\sklearn-env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\a1360\anaconda3\envs\sklearn-env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation of classifier finished in 0.28 s 
Average accuracy: 42.87 % 
Average precision: 28.91 % 
Average recall: 30.49 % 
Average F-Measure: 28.02 % 



c:\Users\a1360\anaconda3\envs\sklearn-env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
# my knn
train_e = tfidf.transform(X)
train_id = list(train.keys())
k = 7
res = {}

for key in test.keys():
    e_test = tfidf.transform([test[key]['claim_text']])
    distance = {}
    simila = cosine_similarity(e_test, train_e)[0]
    for i in range(len(simila)):
        distance[train_id[i]]= simila[i]

    sim = [(k, v) for k, v in sorted(distance.items(), key=lambda item: item[1],reverse=True)][:k]
    sel = [key for key,v in sim]
    temp = {}
    for id in sel:
        label = train[id]['claim_label']
        if label in temp.keys():
            temp[label] += 1
        else:
            temp[label] = 1
    
    best = [label for label in sorted(temp.items(), key=lambda item: item[1],reverse=True)][0][0]
    #print(best)
    res[key] = {'claim_label':best}



SUPPORTS
NOT_ENOUGH_INFO
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
SUPPORTS
NOT_ENOUGH_INFO
NOT_ENOUGH_INFO
SUPPORTS
REFUTES
SUPPORTS
REFUTES
SUPPORTS
SUPPORTS
SUPPORTS
SUPPORTS
SUPPORTS
DISPUTED
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
NOT_ENOUGH_INFO
SUPPORTS
REFUTES
REFUTES
NOT_ENOUGH_INFO
REFUTES
NOT_ENOUGH_INFO
SUPPORTS
DISPUTED
SUPPORTS
NOT_ENOUGH_INFO
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
NOT_ENOUGH_INFO
SUPPORTS
DISPUTED
SUPPORTS
DISPUTED
NOT_ENOUGH_INFO
SUPPORTS
REFUTES
REFUTES
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
DISPUTED
SUPPORTS
NOT_ENOUGH_INFO
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
REFUTES
NOT_ENOUGH_INFO
NOT_ENOUGH_INFO
REFUTES
SUPPORTS
DISPUTED
SUPPORTS
DISPUTED
SUPPORTS
SUPPORTS
SUPPORTS
REFUTES
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
NOT_ENOUGH_INFO
SUPPORTS
NOT_ENOUGH_INFO
SUPPORTS
SUPPORTS
SUPPORTS
NOT_ENOUGH_INFO
NOT_ENOUGH_INFO
SUPPORTS
NOT_ENOUGH_INFO
REFUTES
NOT_ENOUGH_INFO
SUPPORTS
SUPPORTS
REFUTES
SUPPORTS
REFUTES
NOT_ENOUGH_INFO
SUPPORTS
NOT_ENOUGH_INFO
SUPPO

In [88]:
print(res)

{'claim-2967': {'claim_label': 'SUPPORTS'}, 'claim-979': {'claim_label': 'NOT_ENOUGH_INFO'}, 'claim-1609': {'claim_label': 'SUPPORTS'}, 'claim-1020': {'claim_label': 'SUPPORTS'}, 'claim-2599': {'claim_label': 'NOT_ENOUGH_INFO'}, 'claim-2110': {'claim_label': 'SUPPORTS'}, 'claim-1135': {'claim_label': 'SUPPORTS'}, 'claim-712': {'claim_label': 'NOT_ENOUGH_INFO'}, 'claim-1307': {'claim_label': 'SUPPORTS'}, 'claim-148': {'claim_label': 'NOT_ENOUGH_INFO'}, 'claim-903': {'claim_label': 'NOT_ENOUGH_INFO'}, 'claim-2942': {'claim_label': 'SUPPORTS'}, 'claim-1001': {'claim_label': 'REFUTES'}, 'claim-1034': {'claim_label': 'SUPPORTS'}, 'claim-1009': {'claim_label': 'REFUTES'}, 'claim-770': {'claim_label': 'SUPPORTS'}, 'claim-3074': {'claim_label': 'SUPPORTS'}, 'claim-1761': {'claim_label': 'SUPPORTS'}, 'claim-1475': {'claim_label': 'SUPPORTS'}, 'claim-477': {'claim_label': 'SUPPORTS'}, 'claim-1378': {'claim_label': 'DISPUTED'}, 'claim-503': {'claim_label': 'SUPPORTS'}, 'claim-2751': {'claim_label

In [20]:
boost = VotingClassifier(estimators=[
                        ('svc', svm), ('ridge', ridge), ('knn', knn)],
                        voting='hard', n_jobs=-1)

In [21]:
start = timer()
metrics = np.zeros(4)
X_train_gen = [x for x in X_train_cv]
tfidf.fit(X_train_gen)
X_train_vec = tfidf.transform(X_train_gen)
boost.fit(X_train_vec, y_train_cv)
X_test_gen = [x for x in X_test]
X_test_vec = tfidf.transform(X_test_gen)
y_pred = boost.predict(X_test_vec)
metrics = get_metrics(y_test, y_pred, metrics)
dt = timer() - start
metrics = metrics * 100
print('Evaluation of voting classifier on the test set finished in {:.2f} s . \n'
      'Average accuracy: {:.2f} % \n'
      'Average precision: {:.2f} % \n'
      'Average recall: {:.2f} % \n'
      'Average F-Measure: {:.2f} % \n'
      .format(dt, metrics[0], metrics[1],
              metrics[2], metrics[3]))




Evaluation of voting classifier on the test set finished in 2.37 s . 
Average accuracy: 48.37 % 
Average precision: 40.84 % 
Average recall: 37.50 % 
Average F-Measure: 36.86 % 



In [140]:
#prediction

X_test = [x for x in X_test]
#print(len(X_test))
X_test_vec = tfidf.transform(X_test)
y_pred = boost.predict(X_test_vec).tolist()
#print(y_pred)

['SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT_ENOUGH_INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT_ENOUGH_INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT_ENOUGH_INFO', 'NOT_ENOUGH_INFO', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'NOT_ENOUGH_INFO', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'DISPUTED', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT_ENOUGH_INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT_ENOUGH_INFO', 'SUPPO

In [152]:
pre = np.load('test.npy').tolist()


with open('evi_f.json') as json_file:
    format = json.load(json_file)

#with open('sample.json') as json_file:
    #label = json.load(json_file)


counter = 0
for key in format.keys():
    format[key]['claim_label'] = res[key]['claim_label']
    counter +=1

file_path = 'eval_ff.json'
with open(file_path, 'w') as json_file:
    json.dump(format, json_file)